# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [2]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [5]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [6]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

FileNotFoundError: [Errno 2] No such file or directory: 'data/world_bank_projects_less.json'

In [7]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

ValueError: Expected object or value

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [220]:
#Exercise 1
#importing necessary libraries
import pandas as pd

#reading in the json file to a pandas dataframe: df
df=pd.read_json('world_bank_projects.json')

#grouping the dataframe by countryname and counting unique values with .count(): country_count
country_count=df.groupby('countryname')['countryname'].count()


#sorting country_count with sort_values and printing only the top 10
print(country_count.sort_values(ascending=False).head(10))

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
Name: countryname, dtype: int64


In [221]:
#Exercise 3
#creating new dataframe dj_mjcodes that is a copy of the original
df_mjcodes=df

#empty dictionary in which keys will be codes and values with the names
list_of_codes_and_names={}

#looping through all of the elements to determine the name that corresponds to each code 
for row in df_mjcodes['mjtheme_namecode']:
    
    for element in row:
        
        #if the code doesnt already exist and the name isnt empty append to dict
        if (element['code'] not in list_of_codes_and_names and element['name']!=''):
            list_of_codes_and_names[element['code']]=element['name']
            
#looping through again to fill in the empty values using the created dictionary 
for row in df_mjcodes['mjtheme_namecode']:

    for element in row:
        
        #filling the empty name trait using the created dictionary
        if (element['name']==''):
            element['name']=list_of_codes_and_names[element['code']] 
            
#looping through one more time to verify that no missing values exist            
no_missing = True            
for row in df_mjcodes['mjtheme_namecode']:
    
    for element in row:
        
        #if '' is found, setting no _missing to false
        if(element['name']==''):
            no_missing=False
            
#easily understood check
if no_missing ==True:
    print('no missing values')
    
else:
    print('stilling missing some name values')

df_no_missing_values=df_mjcodes
print(df_no_missing_values.head())

no missing values
                                    _id  approvalfy board_approval_month  \
0  {'$oid': '52b213b38594d8a2be17c780'}        1999             November   
1  {'$oid': '52b213b38594d8a2be17c781'}        2015             November   
2  {'$oid': '52b213b38594d8a2be17c782'}        2014             November   
3  {'$oid': '52b213b38594d8a2be17c783'}        2014              October   
4  {'$oid': '52b213b38594d8a2be17c784'}        2014              October   

      boardapprovaldate                                 borrower  \
0  2013-11-12T00:00:00Z  FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA   
1  2013-11-04T00:00:00Z                    GOVERNMENT OF TUNISIA   
2  2013-11-01T00:00:00Z   MINISTRY OF FINANCE AND ECONOMIC DEVEL   
3  2013-10-31T00:00:00Z   MIN. OF PLANNING AND INT'L COOPERATION   
4  2013-10-31T00:00:00Z                      MINISTRY OF FINANCE   

            closingdate                              country_namecode  \
0  2018-07-07T00:00:00Z  Federal Democratic

In [222]:
#Exercise 2
#empty dictionary to hold the names and their asociated counts
names_and_counts={}

#looping throught the DF to count instances of each name
for row in df_no_missing_values['mjtheme_namecode']:
    
    for element in row:
        
        #if the name doesnt exist add to dictionary
        if (element['name'] not in names_and_counts):
            names_and_counts[element['name']]=0
            
        else:
            names_and_counts[element['name']]+=1
            
#creating datafram from names_and_counts for each sorting            
df_sorted=pd.DataFrame(names_and_counts,index=[0])

#switching columns and rows
df_sorted=df_sorted.transpose()

#adding column label
df_sorted.columns=['Count']

#sorting values highest first
df_sorted=df_sorted.sort_values('Count', ascending=False)

#returing top 10 major project themes
print(df_sorted.head(10))

                                              Count
Environment and natural resources management    249
Rural development                               215
Human development                               209
Public sector governance                        198
Social protection and risk management           167
Financial and private sector development        145
Social dev/gender/inclusion                     129
Trade and integration                            76
Urban development                                49
Economic management                              37
